##### Hyperparameter Tuning

Here I will do hyperparameter tuning on the models on the dataset that is balanced and unscaled as that determined the best dataset

Import required packages and libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV

import warnings

Define the evaluate_model function as done in Model training

In [3]:
def evaluate_model(true, predicted, model, X_test):
    precision = precision_score(true, predicted , zero_division = 0)
    recall = recall_score(true, predicted , zero_division = 0)
    f1 = f1_score(true , predicted, zero_division = 0)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(true, y_pred_proba)
    pr_auc = average_precision_score(true, y_pred_proba)
    return precision, recall, f1, roc_auc, pr_auc

Define the no_scale_classification function as defined in Model training

In [ ]:
def no_scale_classification_hyperparam_tuning(X_train, y_train, X_test, y_test):
    
    # Models with hyperparameter tuning
    tuned_models = {
        "Decision Tree": {
            "model": DecisionTreeClassifier(random_state=42),
            "params": {
                'max_depth': [3, 5, 7, 10, 15, 20, None],
                'min_samples_split': [2, 5, 10, 20],
                'min_samples_leaf': [1, 2, 4, 8],
                'criterion': ['gini', 'entropy'],
                'max_features': ['sqrt', 'log2', None],
                'class_weight': ['balanced', None]
            }
        },
        "Random Forest": {
            "model": RandomForestClassifier(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200, 300],
                'max_depth': [5, 10, 15, 20, None],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'max_features': ['sqrt', 'log2'],
                'class_weight': ['balanced', 'balanced_subsample', None],
                'bootstrap': [True, False]
            }
        },
        "XGBoost": {
            "model": XGBClassifier(random_state=42, eval_metric='logloss'),
            "params": {
                'n_estimators': [50, 100, 200, 300],
                'max_depth': [3, 5, 7, 9],
                'learning_rate': [0.01, 0.05, 0.1, 0.2],
                'subsample': [0.6, 0.8, 1.0],
                'colsample_bytree': [0.6, 0.8, 1.0],
                'gamma': [0, 0.1, 0.5, 1],
                'min_child_weight': [1, 3, 5],
                'scale_pos_weight': [1, 2, 3]
            }
        },
        "CatBoost": {
            "model": CatBoostClassifier(random_state=42, verbose=False),
            "params": {
                'iterations': [50, 100, 200, 300],
                'depth': [4, 6, 8, 10],
                'learning_rate': [0.01, 0.05, 0.1, 0.2],
                'l2_leaf_reg': [1, 3, 5, 7],
                'border_count': [32, 64, 128],
                'class_weights': [[1, 1], [1, 2], [1, 3]]
            }
        },
        "AdaBoost": {
            "model": AdaBoostClassifier(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200, 300],
                'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0],
                'algorithm': ['SAMME', 'SAMME.R']
            }
        },
        "GradientBoosting": {
            "model": GradientBoostingClassifier(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200, 300],
                'max_depth': [3, 5, 7, 9],
                'learning_rate': [0.01, 0.05, 0.1, 0.2],
                'subsample': [0.6, 0.8, 1.0],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'max_features': ['sqrt', 'log2', None]
            }
        }
    }
    
    results = []
    
    # Train models with hyperparameter tuning
    print("\nTraining models with hyperparameter tuning...")
    for model_name, model_info in tuned_models.items():
        print(f"  - {model_name} (this may take a while...)")
        
        random_search = RandomizedSearchCV(
            estimator=model_info["model"],
            param_distributions=model_info["params"],
            n_iter=50,
            cv=5,
            scoring='roc_auc',
            random_state=42,
            n_jobs=-1,
            verbose=0
        )
        
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)
        
        accuracy_train = accuracy_score(y_train, y_train_pred)
        accuracy_test = accuracy_score(y_test, y_test_pred)
        
        train_precision, train_recall, train_f1, train_roc_auc, train_pr_auc = evaluate_model(
            y_train, y_train_pred, best_model, X_train
        )
        test_precision, test_recall, test_f1, test_roc_auc, test_pr_auc = evaluate_model(
            y_test, y_test_pred, best_model, X_test
        )
        
        results.append({
            'Model': model_name,
            'Train Accuracy': accuracy_train,
            'Test Accuracy': accuracy_test,
            'Test Precision': test_precision,
            'Test Recall': test_recall,
            'Test F1': test_f1,
            'Test ROC AUC': test_roc_auc,
            'Test PR AUC': test_pr_auc,
            'Tuned': 'Yes',
            'Best Params': str(random_search.best_params_)
        })
    
    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    # Display main results table
    print("\n" + "=" * 150)
    print("MODEL COMPARISON - ALL METRICS")
    print("=" * 150)
    display_df = df_results.drop('Best Params', axis=1)
    print(display_df.to_string(index=False, float_format=lambda x: f'{x:.4f}'))
    
    # Display best parameters for tuned models
    print("\n" + "=" * 150)
    print("BEST HYPERPARAMETERS FOR TUNED MODELS")
    print("=" * 150)
    tuned_df = df_results[df_results['Tuned'] == 'Yes'][['Model', 'Best Params']]
    for idx, row in tuned_df.iterrows():
        print(f"\n{row['Model']}:")
        params = eval(row['Best Params'])
        for param, value in params.items():
            print(f"  - {param}: {value}")
    
    # Display sorted by Test ROC AUC
    print("\n" + "=" * 150)
    print("TOP 5 MODELS BY TEST ROC AUC")
    print("=" * 150)
    top_models = df_results.nlargest(5, 'Test ROC AUC')[['Model', 'Test Accuracy', 'Test ROC AUC', 'Test F1', 'Tuned']]
    print(top_models.to_string(index=False, float_format=lambda x: f'{x:.4f}'))
    print("\n")
    
    return df_results

In [4]:
def no_scale_classification(X_train, y_train, X_test, y_test):

    models = {
        "Decision Tree": DecisionTreeClassifier(),

        "Random Forest Classifier": RandomForestClassifier(),

        "XGBClassifier": XGBClassifier(), 
        
        "CatBoosting Classifier": CatBoostClassifier(verbose=False),

        "AdaBoost Classifier": AdaBoostClassifier(),

        "GradientBoosting Classifier": GradientBoostingClassifier()
    }
 
    # Lists to store results
    results_train = []
    results_test = []
    
    for model_name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Training predictions
        y_train_pred = model.predict(X_train)
        accuracy_train = accuracy_score(y_train, y_train_pred)
        train_precision, train_recall, train_f1, train_roc_auc, train_pr_auc = evaluate_model(
            y_train, y_train_pred, model, X_train
        )
        
        # Test predictions
        y_test_pred = model.predict(X_test)
        accuracy_test = accuracy_score(y_test, y_test_pred)
        test_precision, test_recall, test_f1, test_roc_auc, test_pr_auc = evaluate_model(
            y_test, y_test_pred, model, X_test
        )
        
        # Store training results
        results_train.append({
            'Model': model_name,
            'Accuracy': accuracy_train,
            'Precision': train_precision,
            'Recall': train_recall,
            'F1 Score': train_f1,
            'ROC AUC': train_roc_auc,
            'PR AUC': train_pr_auc
        })
        
        # Store test results
        results_test.append({
            'Model': model_name,
            'Accuracy': accuracy_test,
            'Precision': test_precision,
            'Recall': test_recall,
            'F1 Score': test_f1,
            'ROC AUC': test_roc_auc,
            'PR AUC': test_pr_auc
        })
    
    # Create DataFrames
    df_train = pd.DataFrame(results_train)
    df_test = pd.DataFrame(results_test)
    
    # Display tables
    print("=" * 120)
    print("TRAINING SET PERFORMANCE")
    print("=" * 120)
    print(df_train.to_string(index=False, float_format=lambda x: f'{x:.4f}'))
    print("\n")
    
    print("=" * 120)
    print("TEST SET PERFORMANCE")
    print("=" * 120)
    print(df_test.to_string(index=False, float_format=lambda x: f'{x:.4f}'))
    print("\n")
    

Load in the unscaled and balanced dataset

In [5]:
X_train=pd.read_csv('data/unscaled_balanced_X_train.csv')
y_train=pd.read_csv('data/unscaled_balanced_y_train.csv')

Load in the test datasets

In [6]:
X_test=pd.read_csv('data/X_test.csv')
y_test=pd.read_csv('data/y_test.csv')